# <h1 align="center"><font color="red">ChatBot usando qdrant</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook segue o tutorial de [Prince Krampah]()

O dataset que usaremos pode ser encontrado no [kaggle: Customer Support Ticket Dataset](https://www.kaggle.com/datasets/suraj520/customer-support-ticket-dataset).

https://github.com/Princekrampah/qdrant_csv_chatbot_tutorial/blob/master/data_preprocessing.ipynb


https://medium.com/@princekrampah/building-a-smart-chatbot-using-qdrant-cloud-and-langchain-for-customer-support-tickets-504a3e17ac83

In [1]:
import pandas as pd

df = pd.read_csv("./data/customer_support_tickets.csv")
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [2]:
print(df["Ticket Description"].iloc[0])

I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [4]:
df.shape

(8469, 17)

In [5]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [6]:
product_ticket_description_df = df[['Product Purchased', 'Ticket Description']].reset_index().rename(columns={'index': 'ID'})

In [7]:
product_ticket_description_df.head()

,ID,Product Purchased,Ticket Description
0,0,GoPro Hero,I'm having an issue with the {product_purchase...
1,1,LG Smart TV,I'm having an issue with the {product_purchase...
2,2,Dell XPS,I'm facing a problem with my {product_purchase...
3,3,Microsoft Office,I'm having an issue with the {product_purchase...
4,4,Autodesk AutoCAD,I'm having an issue with the {product_purchase...


In [8]:
import textwrap

print(product_ticket_description_df["Ticket Description"].iloc[0])

#ou
#print(textwrap.fill(product_ticket_description_df["Ticket Description"].iloc[0], width=250))

I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.


In [9]:
# Create a new column with the product name inserted in the ticket description:
product_ticket_description_df['Updated Ticket Description'] = df.apply(
    lambda row: row['Ticket Description'].replace('{product_purchased}', row['Product Purchased']),
    axis=1
)

In [10]:
product_ticket_description_df.head()

,ID,Product Purchased,Ticket Description,Updated Ticket Description
0,0,GoPro Hero,I'm having an issue with the {product_purchase...,I'm having an issue with the GoPro Hero. Pleas...
1,1,LG Smart TV,I'm having an issue with the {product_purchase...,I'm having an issue with the LG Smart TV. Plea...
2,2,Dell XPS,I'm facing a problem with my {product_purchase...,I'm facing a problem with my Dell XPS. The Del...
3,3,Microsoft Office,I'm having an issue with the {product_purchase...,I'm having an issue with the Microsoft Office....
4,4,Autodesk AutoCAD,I'm having an issue with the {product_purchase...,I'm having an issue with the Autodesk AutoCAD....


In [11]:
print(product_ticket_description_df["Ticket Description"].iloc[0])

I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.


In [12]:
print(product_ticket_description_df["Updated Ticket Description"].iloc[0])

I'm having an issue with the GoPro Hero. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.


In [13]:
product_ticket_description_df = product_ticket_description_df[['ID', 'Product Purchased', 'Updated Ticket Description']]
product_ticket_description_df.head()

,ID,Product Purchased,Updated Ticket Description
0,0,GoPro Hero,I'm having an issue with the GoPro Hero. Pleas...
1,1,LG Smart TV,I'm having an issue with the LG Smart TV. Plea...
2,2,Dell XPS,I'm facing a problem with my Dell XPS. The Del...
3,3,Microsoft Office,I'm having an issue with the Microsoft Office....
4,4,Autodesk AutoCAD,I'm having an issue with the Autodesk AutoCAD....


In [14]:
product_ticket_description_df.shape

(8469, 3)

In [15]:
product_ticket_description_df.rename(columns={'Updated Ticket Description': 'ticket_description',
                                              'Product Purchased': 'product_purchased'
                                             }, inplace=True)

In [16]:
product_ticket_description_df.head()

,ID,product_purchased,ticket_description
0,0,GoPro Hero,I'm having an issue with the GoPro Hero. Pleas...
1,1,LG Smart TV,I'm having an issue with the LG Smart TV. Plea...
2,2,Dell XPS,I'm facing a problem with my Dell XPS. The Del...
3,3,Microsoft Office,I'm having an issue with the Microsoft Office....
4,4,Autodesk AutoCAD,I'm having an issue with the Autodesk AutoCAD....


In [17]:
product_ticket_description_df.iloc[0].ticket_description

"I'm having an issue with the GoPro Hero. Please assist.\n\nYour billing zip code is: 71701.\n\nWe appreciate that you have requested a website address.\n\nPlease double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists."

In [18]:
product_ticket_description_df["ticket_description"] = product_ticket_description_df.apply(
    lambda row: row.ticket_description.strip().replace('\n', '').replace('\n', ''),
    axis=1
)

In [19]:
product_ticket_description_df.iloc[0].ticket_description

"I'm having an issue with the GoPro Hero. Please assist.Your billing zip code is: 71701.We appreciate that you have requested a website address.Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists."

In [20]:
print(product_ticket_description_df.iloc[0].ticket_description)

I'm having an issue with the GoPro Hero. Please assist.Your billing zip code is: 71701.We appreciate that you have requested a website address.Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.


In [21]:
product_ticket_description_df["product_purchased"] = product_ticket_description_df.apply(lambda row: row.product_purchased.strip(), axis=1)

In [22]:
print(product_ticket_description_df.iloc[0].product_purchased)

GoPro Hero


In [23]:
product_ticket_description_df.shape

(8469, 3)

In [24]:
product_ticket_description_df.head()

,ID,product_purchased,ticket_description
0,0,GoPro Hero,I'm having an issue with the GoPro Hero. Pleas...
1,1,LG Smart TV,I'm having an issue with the LG Smart TV. Plea...
2,2,Dell XPS,I'm facing a problem with my Dell XPS. The Del...
3,3,Microsoft Office,I'm having an issue with the Microsoft Office....
4,4,Autodesk AutoCAD,I'm having an issue with the Autodesk AutoCAD....


In [25]:
# Salvando o DataFrame no CSV:
#product_ticket_description_df.to_csv("./data/product_ticket_description.csv", index=False)

# <font color="gree">Criando os Embeddings</font>

In [26]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
#Eddy_key_openai  = os.environ['OPENAI_API_KEY']

#from openai import OpenAI
#client = OpenAI(api_key=Eddy_key_openai)

In [28]:
# %pip install -qU langchain-openai python-decouple

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large", # Com a classe de modelos 'text-embedding-3', você pode especificar o tamanho dos embeddings que deseja retornar.
    dimensions= 1536 # 3072
)

In [29]:
df = pd.read_csv("./data/product_ticket_description.csv")

df.head()

,ID,product_purchased,ticket_description
0,0,GoPro Hero,I'm having an issue with the GoPro Hero. Pleas...
1,1,LG Smart TV,I'm having an issue with the LG Smart TV. Plea...
2,2,Dell XPS,I'm facing a problem with my Dell XPS. The Del...
3,3,Microsoft Office,I'm having an issue with the Microsoft Office....
4,4,Autodesk AutoCAD,I'm having an issue with the Autodesk AutoCAD....


In [30]:
df = df[:250]
df.shape

(250, 3)

In [31]:
df["product_purchased_embeddings"] = df.apply(
    lambda row: list(embeddings.embed_query(row.product_purchased)),
    axis=1
)


In [32]:
df.dtypes

ID                               int64
product_purchased               object
ticket_description              object
product_purchased_embeddings    object
dtype: object

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ID                            250 non-null    int64 
 1   product_purchased             250 non-null    object
 2   ticket_description            250 non-null    object
 3   product_purchased_embeddings  250 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.9+ KB
